In [35]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
import IPython.display as ipd
import pandas as pd

In [36]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [37]:
  SAMPLING_RATE = 16000
  DURATION = 10
  DATA_PER_CLASS = 60*40*SAMPLING_RATE #40 mins of data per class

In [38]:
A, _ = librosa.load('/content/drive/MyDrive/val_set/speech_and_music/music_long.wav', duration=600, sr=16000)
B, _ = librosa.load('/content/drive/MyDrive/val_set/speech_and_music/speech_indian.wav', duration=600, sr=16000)
C, _ = librosa.load('/content/drive/MyDrive/val_set/speech_and_music/speech_indian.wav', offset=960, duration=600, sr=16000)
D, _ = librosa.load('/content/drive/MyDrive/val_set/speech_and_music/speech_indian.wav', offset=34*60, duration=600, sr=16000)
E, _ = librosa.load('/content/drive/MyDrive/val_set/speech_and_music/Spiritual.wav', duration=1200, sr=16000)
F, _ = librosa.load('/content/drive/MyDrive/val_set/speech_and_music/AnneHathway.wav', duration=600, sr=16000)
G, _ = librosa.load('/content/drive/MyDrive/val_set/speech_and_music/RockMusic.wav', duration=600, sr=16000)

Atif_Aslam = A
S_Trivedi = B
AudioBook = C
R_Madhavan = D
Hanuman_Chalisa = E
Anne_Hathway = F
Rock_Music = G

In [39]:
# A, _ = librosa.load('/content/drive/MyDrive/val_set/speech_and_music/speech_indian.wav', sr=SAMPLING_RATE)
# A = A[:DATA_PER_CLASS]
A = np.concatenate([S_Trivedi, R_Madhavan, Anne_Hathway, AudioBook])
print(A.shape)

(38400000,)


In [40]:
# B, _ = librosa.load('/content/drive/MyDrive/val_set/speech_and_music/music_long.wav', sr=SAMPLING_RATE)
# B = B[:DATA_PER_CLASS]
B = np.concatenate([Atif_Aslam, Hanuman_Chalisa, Rock_Music])
print(B.shape)

(38400000,)


In [41]:
C = np.zeros((DATA_PER_CLASS))
print(C.shape)

(38400000,)


In [45]:
noise = np.random.rand((DURATION*SAMPLING_RATE))
noise = noise*2
noise = noise - 1

In [47]:
X_speech = A
X_speech = np.resize(X_speech, (-1, DURATION*SAMPLING_RATE))
y_speech = np.array(['speech']*len(X_speech))
print(X_speech.shape)

(239, 160000)


In [48]:
X_music = B
X_music = np.resize(X_music, (-1, DURATION*SAMPLING_RATE))
y_music = np.array(['music']*len(X_music))
print(X_music.shape)

(239, 160000)


In [49]:
SILENCE_SIZE = C.shape[0]
endpt = (DURATION*SAMPLING_RATE)*int(SILENCE_SIZE/(DURATION*SAMPLING_RATE))
X_silence = C[:endpt]
X_silence = np.resize(X_silence, (-1, DURATION*SAMPLING_RATE))
y_silence = np.array(['silence']*len(X_silence))
print(X_silence.shape)

(239, 160000)


In [50]:
# Randomly add white noise
X = np.concatenate([X_speech, X_music, X_silence])
y = np.concatenate([y_speech, y_music, y_silence])

for i in range(X.shape[0]):
    # Adding 0 to 3 percent noise randomly to each sound
    X[i] = X[i] + noise*(np.random.randint(3)/100)


In [ ]:
from sklearn.utils import shuffle
X, y = shuffle(X, y)

In [ ]:
import IPython.display as ipd

In [51]:
print(y[243])

music


In [53]:
ipd.Audio(X[243], rate=SAMPLING_RATE)

In [ ]:
np.save('/content/drive/MyDrive/val_set/processed_data/X', X)

In [ ]:
np.save('/content/drive/MyDrive/val_set/processed_data/y', y)

In [ ]:
X = np.load('/content/drive/MyDrive/val_set/processed_data/X.npy')

In [54]:
def calc_spec(x):
    n_fft = 1024
    hop_length = 512
    win_length = 1024
    X = np.abs(librosa.stft(x, n_fft = n_fft, hop_length = hop_length, win_length = win_length, window='hann', dtype = np.complex256))
    X = librosa.power_to_db(X**2,ref=np.max)
    return X

In [55]:
from tqdm import tqdm

In [56]:
def convert2spect(X):
  X_spectrograms = []
  for i in tqdm(X):
    X_spectrograms.append(calc_spec(i))
  return np.array(X_spectrograms)

In [57]:
A = convert2spect(X)
print(A.shape)

100%|██████████| 717/717 [00:24<00:00, 29.13it/s]


(717, 513, 313)


In [58]:
np.save('/content/drive/MyDrive/val_set/Final_Dataset/training_spectrogram.npy', A)

In [59]:
np.save('/content/drive/MyDrive/val_set/Final_Dataset/training_labels.npy', y)

In [ ]:
print(X.shape)

(717, 160000)
